In [1]:
from __future__ import print_function
import datetime
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import json

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

def authenticate_google_calendar():
    """
    Authenticate and create a Google Calendar API service.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('calendar', 'v3', credentials=creds)
    return service

def fetch_study_events(service, calendar_id='primary'):
    """
    Fetches study events from a given calendar.
    """
    now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
    print('Fetching the upcoming 10 events')
    events_result = service.events().list(calendarId=calendar_id, timeMin=now,
                                          maxResults=10, singleEvents=True,
                                          orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print('No upcoming events found.')
        return []

    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        print(start, event['summary'])

    return events

def calculate_study_stats(events):
    """
    Calculate total study time and other stats from events.
    """
    total_study_time = 0
    # Example calculation, adapt according to event structure
    for event in events:
        start = datetime.datetime.fromisoformat(event['start'].get('dateTime', event['start'].get('date')))
        end = datetime.datetime.fromisoformat(event['end'].get('dateTime', event['end'].get('date')))
        duration = (end - start).total_seconds() / 3600  # Convert to hours
        total_study_time += duration

    print(f"Total study time: {total_study_time} hours")
    # Add more stats as needed

def create_study_plan(service, subjects, calendar_id='primary'):
    """
    Create a study plan based on subjects and available time slots.
    """
    # Placeholder for logic to create a study plan.
    # You need to define how to find gaps and allocate subjects based on weights.
    print("Creating study plan...")
    # Implement your logic here


In [2]:

service = authenticate_google_calendar()
events = fetch_study_events(service, '0ca09266015f691eebe0d00c6f3ed7a784713e0160a694b8f7929add00cb1aa1@group.calendar.google.com')  # Specify your calendar ID for "studying"
calculate_study_stats(events)
subjects = {'Math': 2, 'Physics': 3, 'Chemistry': 1}  # Example subjects with weights
create_study_plan(service, subjects, 'studying')  # Specify your calendar ID


FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'